In [1]:
import pandas as pd
import csv

## Data collection

In [2]:
tables = pd.read_html(
    "https://en.wikipedia.org/wiki/List_of_deaths_on_eight-thousanders"
)
everest_tables = pd.read_html(
    "https://en.wikipedia.org/wiki/List_of_people_who_died_climbing_Mount_Everest"
)
print("Len tables", len(tables))
print("Len everest_tables", len(everest_tables))

Len tables 28
Len everest_tables 10


In [3]:
everest_tables[1].columns

Index(['Name', 'Date', 'Age', 'Expedition', 'Nationality', 'Cause of death',
       'Location', 'Remains Status', 'Refs'],
      dtype='object')

## Creating Dataframe

In [4]:
valid_table_idxs = [2, 4, 6, 8, 9, 11, 13, 15, 17, 19, 21, 23, 24]
idx_to_mountain_lookup = {
    2: "K2",
    4: "Kangchenjunga",
    6: "Lhotse",
    8: "Makalu",
    9: "Cho Oyu",
    11: "Dhaulagiri I",
    13: "Manaslu",
    15: "Nanga Parbat",
    17: "Annapurna I",
    19: "Gasherbrum I",
    21: "Broad Peak",
    23: "Gasherbrum II",
    24: "Shishapangma",
}

In [5]:
all_mountains = pd.DataFrame()

for idx in valid_table_idxs:
    table = tables[idx]
    table["Mountain"] = idx_to_mountain_lookup[idx]

    if all_mountains.empty:
        all_mountains = table
        continue
    all_mountains = pd.concat([all_mountains, table], ignore_index=True)

all_mountains.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 749 entries, 0 to 748
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Date            749 non-null    object
 1   Name            749 non-null    object
 2   Nationality     749 non-null    object
 3   Cause of death  749 non-null    object
 4   -               96 non-null     object
 5   Mountain        749 non-null    object
 6   References      615 non-null    object
dtypes: object(7)
memory usage: 41.1+ KB


#### Adding everest data to the all mountains table


In [6]:
everest_useful_df = everest_tables[1][
    ["Name", "Date", "Nationality", "Cause of death"]
].copy()
everest_useful_df["Mountain"] = "Mount Everest"

all_mountains = pd.concat([all_mountains, everest_useful_df], ignore_index=True)
all_mountains.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1081 entries, 0 to 1080
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Date            1081 non-null   object
 1   Name            1081 non-null   object
 2   Nationality     1078 non-null   object
 3   Cause of death  1072 non-null   object
 4   -               96 non-null     object
 5   Mountain        1081 non-null   object
 6   References      615 non-null    object
dtypes: object(7)
memory usage: 59.2+ KB


## Data Preprocessing

In [7]:
all_mountains.drop(columns=["-", "References"], inplace=True)

In [11]:
all_mountains["Date"] = all_mountains["Date"].apply(lambda x: x.split("/")[-1])
all_mountains["Date"] = all_mountains["Date"].apply(lambda x: x.split("–")[-1])
all_mountains["Date"] = all_mountains["Date"].apply(lambda x: x.split("or")[0])
all_mountains["Date"] = all_mountains["Date"].apply(lambda x: x.split("[")[0])

values_to_replace = ["Autumn", "Spring", "Winter", "Summer", "mid", "late", "early"]
for value in values_to_replace:
    all_mountains["Date"] = all_mountains["Date"].str.replace(value, "").str.strip()

all_mountains["Date"] = pd.to_datetime(all_mountains["Date"], format="mixed")

In [13]:
all_mountains.drop_duplicates(inplace=True)

In [16]:
all_mountains.fillna(pd.NA, inplace=True)
all_mountains.isna().sum()

Date              0
Name              0
Nationality       2
Cause of death    9
Mountain          0
dtype: int64

In [15]:
all_mountains.sample(10)

,Date,Name,Nationality,Cause of death,Mountain
73,1986-08-10,Hannes Wieser,Austria,Altitude sickness,K2
903,1997-05-07,Nikolai Chevtchenko,Russia,Exhaustion and/or exposure,Mount Everest
292,2018-09-27,Dawa Gyaljen Sherpa,Nepal,Avalanche,Dhaulagiri I
303,2013-05-23,Chizuko Kono,Japan,Exhaustion,Dhaulagiri I
725,2009-10-15,Roby Piantoni,Italy,Fall,Shishapangma
672,2006-07-08,Markus Kronthaler,Austria,Exhaustion,Broad Peak
491,1996-06-20,Gabriel Stana,Romania,Avalanche,Nanga Parbat
869,1993-04-23,Pasang Lhamu Sherpa,Nepal,Disappearance (likely accidental death during ...,Mount Everest
323,2000-10-07,Gyalzen Chuldim,Nepal,Avalanche,Dhaulagiri I
495,1990-07-03,Chang-gi Park,South Korea,Fall into a crevasse,Nanga Parbat


In [18]:
all_mountains.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1078 entries, 0 to 1080
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Date            1078 non-null   datetime64[ns]
 1   Name            1078 non-null   object        
 2   Nationality     1076 non-null   object        
 3   Cause of death  1069 non-null   object        
 4   Mountain        1078 non-null   object        
dtypes: datetime64[ns](1), object(4)
memory usage: 50.5+ KB


## Final dataset

In [ ]:
all_mountains.to_csv(
    "deaths_on_eight-thousanders.csv", index=False, quoting=csv.QUOTE_NONNUMERIC
)